# The Blue Sky Challenge  
 **Team**

 **PYPI_PLAYERS**

 **Teammates**

- Owais Ahmad [PortFolio](https://owaiskhan9654.github.io/)

- Ahmad Mujtaba [PortFolio](https://www.linkedin.com/in/ahmad-mujtaba-4198b31b8/)







**Refrences**

- K. Thiyagarajan, S. Kodagoda, L. Van Nguyen and R. Ranasinghe, "Sensor Failure Detection and Faulty Data Accommodation Approach for Instrumented Wastewater Infrastructures," in IEEE Access, vol. 6, pp. 56562-56574, 2018, 
doi: [10.1109/ACCESS.2018.2872506](https://ieeexplore.ieee.org/document/8476572).

- A Python interface for the netCDF4 file-format that reads and writes local or remote HDF5 files directly via h5py or h5pyd, without relying on the Unidata netCDF library. https://github.com/h5netcdf/h5netcdf

### Part - 2

In [1]:
from netCDF4 import Dataset
import numpy as np
import netCDF4 as nc4

from sentinelsat import SentinelAPI, geojson_to_wkt
import json
from tqdm import tqdm

In [2]:
nc_file = 'K:\\Blue Sky Challenge\\BlueSkyChallenge-main\\data\\L2\\S5P_OFFL_L2__NO2____20210325T104955_20210325T123125_17863_01_010400_20210327T044257.nc'
Data = Dataset(nc_file, mode='r')

In [3]:
Data.groups['PRODUCT'].variables

{'scanline': <class 'netCDF4._netCDF4.Variable'>
 int32 scanline(scanline)
     units: 1
     axis: Y
     long_name: along-track dimension index
     comment: This coordinate variable defines the indices along track; index starts at 0
     _FillValue: -2147483647
 path = /PRODUCT
 unlimited dimensions: 
 current shape = (3735,)
 filling on,
 'ground_pixel': <class 'netCDF4._netCDF4.Variable'>
 int32 ground_pixel(ground_pixel)
     units: 1
     axis: X
     long_name: across-track dimension index
     comment: This coordinate variable defines the indices across track, from west to east; index starts at 0
     _FillValue: -2147483647
 path = /PRODUCT
 unlimited dimensions: 
 current shape = (450,)
 filling on,
 'time': <class 'netCDF4._netCDF4.Variable'>
 int32 time(time)
     units: seconds since 2010-01-01 00:00:00
     standard_name: time
     axis: T
     long_name: reference time for the measurements
     comment: The time in this variable corresponds to the time in the time_refer

In [4]:
lons = Data.groups['PRODUCT'].variables['longitude'][:][0,:,:]
lats = Data.groups['PRODUCT'].variables['latitude'][:][0,:,:]
no2 = Data.groups['PRODUCT'].variables['nitrogendioxide_tropospheric_column_precision'][0,:,:]
print(lons.shape)
print(lats.shape)
print(no2.shape)

(3735, 450)
(3735, 450)
(3735, 450)


In [5]:
LatMin=51.25
LatMax=51.75
LngMin=-0.6
LngMax=0.28

In [6]:
latbounds = [ 51.25 , 51.75 ]
lonbounds = [ -0.6 ,0.28 ] # degrees east ? 
#lats = f.variables['latitude'][:] 
#lons = f.variables['longitude'][:]

# latitude lower and upper index
latli = np.argmin( np.abs( lats - latbounds[0] ) )
latui = np.argmin( np.abs( lats - latbounds[1] ) ) 

# longitude lower and upper index
lonli = np.argmin( np.abs( lons - lonbounds[0] ) )
lonui = np.argmin( np.abs( lons - lonbounds[1] ) ) 

In [7]:
print(Data.groups['PRODUCT'].variables['nitrogendioxide_tropospheric_column_precision'][ : , latli:latui , lonli:lonui])

[]


In [8]:
print(Data.groups['PRODUCT'].variables['nitrogendioxide_tropospheric_column_precision'][:,:,:])

[[[-- -- -- ... -- -- --]
  [-- -- -- ... -- -- --]
  [-- -- -- ... -- -- --]
  ...
  [-- 6.032328656147001e-06 6.1571176956931595e-06 ...
   2.8904687496833503e-05 2.9103593988111243e-05 3.01687450701138e-05]
  [-- 6.141295671113767e-06 6.262668648560066e-06 ...
   2.9375014491961338e-05 3.0133131076581776e-05 3.123654460068792e-05]
  [-- 6.421447324100882e-06 6.110448339313734e-06 ...
   2.973610389744863e-05 3.17246958729811e-05 3.134011785732582e-05]]]


In [9]:
import glob

allFiles = glob.glob('BlueSkyChallenge-main\data\L2\S5P_OFFL_L2__NO2*') 
allFiles = [i for i in allFiles if '.nc' in i[-4:len(i)]]
allFiles

['BlueSkyChallenge-main\\data\\L2\\S5P_OFFL_L2__NO2____20210325T104955_20210325T123125_17863_01_010400_20210327T044257.nc',
 'BlueSkyChallenge-main\\data\\L2\\S5P_OFFL_L2__NO2____20210325T123125_20210325T141256_17864_01_010400_20210327T060331.nc',
 'BlueSkyChallenge-main\\data\\L2\\S5P_OFFL_L2__NO2____20210326T103100_20210326T121231_17877_01_010400_20210328T041830.nc',
 'BlueSkyChallenge-main\\data\\L2\\S5P_OFFL_L2__NO2____20210326T121231_20210326T135401_17878_01_010400_20210328T052956.nc',
 'BlueSkyChallenge-main\\data\\L2\\S5P_OFFL_L2__NO2____20210327T101205_20210327T115336_17891_01_010400_20210329T033009.nc',
 'BlueSkyChallenge-main\\data\\L2\\S5P_OFFL_L2__NO2____20210327T115336_20210327T133506_17892_01_010400_20210329T052835.nc',
 'BlueSkyChallenge-main\\data\\L2\\S5P_OFFL_L2__NO2____20210328T113441_20210328T131611_17906_01_010400_20210330T050349.nc',
 'BlueSkyChallenge-main\\data\\L2\\S5P_OFFL_L2__NO2____20210329T111545_20210329T125716_17920_01_010400_20210331T045425.nc',
 'BlueSk

In [10]:
for file in allFiles:
    Data = Dataset(file, mode='r')
    lons = np.around(Data.groups['PRODUCT'].variables['longitude'][:][0,:,:],)
    lats = np.around(Data.groups['PRODUCT'].variables['latitude'][:][0,:,:],)
    no2 = Data.groups['PRODUCT'].variables['nitrogendioxide_tropospheric_column_precision'][0,:,:]
    latbounds = [ 51.25 , 51.75 ]
    lonbounds = [ -0.6 ,  0.28 ]
    
    where_j = np.where((lons >= -0.6) & (lons <= 0.28))[0]
    where_i = np.where((lats >= 51.25) & (lats <= 52.75))[0]

    i0 = where_i[0]
    i1 = where_i[-1]+1

    j0 = where_j[0]
    j1 = where_j[-1]+1
 
    print(Data.groups['PRODUCT'].variables['nitrogendioxide_tropospheric_column_precision'][ : , i0:i1 , j0:j1])
    print('\n\n\n\n\n\n\n\n')
    

[[[3.899702278431505e-05 3.4818356652976945e-05 3.014190042449627e-05
   ... 1.770283779478632e-05 1.859555959526915e-05
   1.8350372556596994e-05]
  [3.213895615772344e-05 2.6409863494336605e-05 2.256683546875138e-05
   ... 1.7019609003909864e-05 1.915726716106292e-05
   1.758873440849129e-05]
  [4.1003597289090976e-05 3.508227382553741e-05 2.752851287368685e-05
   ... 1.7363478036713786e-05 1.7571006537764333e-05 1.6448926544399e-05]
  ...
  [1.0568122888798825e-05 1.2023917406622786e-05 1.6513713489985093e-05
   ... 3.247497807024047e-05 3.3085121685871854e-05
   3.1804174795979634e-05]
  [6.175707312650047e-06 8.698693818587344e-06 1.598603012098465e-05 ...
   3.116072548436932e-05 3.0279783459263854e-05 3.267439751653001e-05]
  [1.4290164472186007e-05 1.373491068079602e-05 1.1612343769229483e-05
   ... 3.2225616450887173e-05 2.9833881853846833e-05
   3.1087394745554775e-05]]]









[[[9.473653335589916e-06 9.020953257277142e-06 9.005041647469625e-06 ...
   4.463803270482458e-05

[[[1.2206259270897135e-05 1.2187233551230747e-05 1.19022433864302e-05
   ... 3.977158121415414e-05 4.4678057747660205e-05
   3.620886491262354e-05]
  [1.269511540158419e-05 1.2869341844634619e-05 1.2607535609276965e-05
   ... 3.881411976180971e-05 3.741748514585197e-05 2.954375850094948e-05]
  [1.3406947800831404e-05 1.3642839803651441e-05 1.3130827028362546e-05
   ... 3.8465321267722175e-05 2.9628301490447484e-05
   3.105363066424616e-05]
  ...
  [5.155224243935663e-06 6.395787295332411e-06 6.593354100914439e-06 ...
   5.2551728003891185e-05 4.966166306985542e-05 5.306769162416458e-05]
  [7.248032034112839e-06 5.54043208467192e-06 7.1288841354544275e-06 ...
   5.232429612078704e-05 5.013918053009547e-05 5.249666719464585e-05]
  [5.614727342617698e-06 5.176154445507564e-06 7.660992196178995e-06 ...
   5.1185230404371396e-05 5.382277595344931e-05 5.3103914979146793e-05]]]









[[[1.7458532965974882e-05 2.3919363229651935e-05 1.8421971617499366e-05
   ... 1.841588891693391e-05 1.7914

In [12]:
Data.groups['PRODUCT'].variables['nitrogendioxide_tropospheric_column_precision'][ : , i0:i1 , j0:j1]

masked_array(
  data=[[[8.229101695178542e-06, 7.968480531417299e-06,
          8.834833352011628e-06, ..., 3.81708778149914e-05,
          3.279999873484485e-05, 4.8081608838401735e-05],
         [7.083328000589972e-06, 7.04890499036992e-06,
          7.717581866018008e-06, ..., 3.424454553169198e-05,
          3.640961949713528e-05, 4.220308983349241e-05],
         [7.892054782132618e-06, 9.715479791339021e-06,
          7.5881739576288965e-06, ..., 3.165398447890766e-05,
          3.375910091563128e-05, 4.858606916968711e-05],
         ...,
         [5.2811310524703e-06, 5.576460807787953e-06,
          5.193603101361077e-06, ..., 9.519022569293156e-06,
          9.385366865899414e-06, 1.0861604096135125e-05],
         [5.344407327356748e-06, 6.061688054614933e-06,
          6.111782568041235e-06, ..., 9.869064342638012e-06,
          9.639688869356178e-06, 9.96173002931755e-06],
         [5.339687049854547e-06, 6.138142907730071e-06,
          5.017170224164147e-06, ..., 1.00070974

In [ ]:
Data.groups['PRODUCT'].variables['nitrogendioxide_tropospheric_column_precision'][ : , i0:i1 , j0:j1]

In [107]:
def nc_to_df(ncfile):
    """
    Purpose: This converts a TROPOMI NO2 file to a Pandas DataFrame.

    Notes:
    This was adapted from read_tropomi_no2_and_dump_ascii.py by Justin Roberts-Pierel, 2015 
    of NASA ARSET.
    
    Parameters:
    ncfile: NetCD4 file from Copernicus S5P Open Data Hub

    Returns:
    dataframe: data from NetCD4 file
    """
    
    try:
        f = open(ncfile, 'r')
    except OSError:
        print('cannot open', ncfile)
    
    
    df = pd.DataFrame()
        
    # read the data
    if 'NO2___' in ncfile and 'S5P' in ncfile:
        tic = time.perf_counter()
        FILENAME = ncfile
        print(ncfile+' is a TROPOMI NO2 file.')

        #this is how you access the data tree in an NetCD4 file
        SDS_NAME='nitrogendioxide_tropospheric_column'
        file = Dataset(ncfile,'r')
        grp='PRODUCT' 
        ds=file
        grp='PRODUCT'
        lat= ds.groups[grp].variables['latitude'][0][:][:]
        lon= ds.groups[grp].variables['longitude'][0][:][:]
        data= ds.groups[grp].variables[SDS_NAME]      
        #get necessary attributes 
        fv=data._FillValue

        #get scan time and turn it into a vector
        scan_time= ds.groups[grp].variables['time_utc']
        # scan_time=geolocation['Time'][:].ravel()

        year = np.zeros(lat.shape)
        mth = np.zeros(lat.shape)
        doy = np.zeros(lat.shape)
        hr = np.zeros(lat.shape)
        mn = np.zeros(lat.shape)
        sec = np.zeros(lat.shape)
        strdatetime = np.zeros(lat.shape)

        for i in range(0,lat.shape[0]):
            t = scan_time[0][i].split('.')[0]
            t1 = t.replace('T',' ')
            t2 = dt.datetime.strptime(t,'%Y-%m-%dT%H:%M:%S')
            t3 = t2.strftime("%S")
            #y = t2.year
            #m = t2.month
            #d = t2.day
            #h = t2.hour
            #m = t2.minute
            #s = t2.second

            #year[i][:] = y
            #mth[i][:] = m
            #doy[i][:] = d
            #hr[i][:] = h
            #mn[i][:] = m
            #sec[i][:] = s
            strdatetime[i][:] = t3
        vlist = list(file[grp].variables.keys())
        #df['Year'] = year.ravel()
        #df['Month'] = mth.ravel()
        #df['Day'] = doy.ravel()
        #df['Hour'] = hr.ravel()
        #df['Minute'] = mn.ravel()
        #df['Second'] = sec.ravel()
        df['UnixTimestamp'] = strdatetime.ravel()
        df['DateTime'] = pd.to_datetime(df['UnixTimestamp'], unit='s')
        df[['Date','Time']] = df['DateTime'].astype(str).str.split(' ',expand=True)
        # This for loop saves all of the SDS in the dictionary at the top 
        #    (dependent on file type) to the array (with titles)
        for i in range(0,len(vlist)):
            SDS_NAME=vlist[(i)] # The name of the sds to read
            #get current SDS data, or exit program if the SDS is not found in the file
            #try:
            sds=ds.groups[grp].variables[SDS_NAME]
            if len(sds.shape) == 3:
                print(SDS_NAME,sds.shape)
                # get attributes for current SDS
                if 'qa' in SDS_NAME:
                    scale=sds.scale_factor
                else: scale = 1.0
                fv=sds._FillValue

                # get SDS data as a vector
                data=sds[:].ravel()
                # The next few lines change fill value/missing value to NaN so 
                #     that we can multiply valid values by the scale factor, 
                #     then back to fill values for saving
                data=data.astype(float)
                data=(data)*scale
                data[np.isnan(data)]=fv
                data[data==float(fv)]=np.nan

                df[SDS_NAME] = data
        toc = time.perf_counter()
        elapsed_time = toc-tic
        print("Processed "+ncfile+" in "+str(elapsed_time/60)+" minutes")
    else:
        raise NameError('Not a TROPOMI NO2 file name.')

    return df

In [110]:
df

,UnixTimestamp,DateTime,Date,Time,latitude,longitude,qa_value,nitrogendioxide_tropospheric_column,nitrogendioxide_tropospheric_column_precision,nitrogendioxide_tropospheric_column_precision_kernel,air_mass_factor_troposphere,air_mass_factor_total,tm5_tropopause_layer_index
0,30.0,1970-01-01 00:00:30,1970-01-01,00:00:30,-80.200417,-121.122574,0.0000,9.969210e+36,9.969210e+36,9.969210e+36,9.969210e+36,9.969210e+36,-2.147484e+09
1,30.0,1970-01-01 00:00:30,1970-01-01,00:00:30,-80.245056,-121.573227,0.0000,9.969210e+36,9.969210e+36,9.969210e+36,9.969210e+36,9.969210e+36,-2.147484e+09
2,30.0,1970-01-01 00:00:30,1970-01-01,00:00:30,-80.288475,-122.021530,0.0000,9.969210e+36,9.969210e+36,9.969210e+36,9.969210e+36,9.969210e+36,-2.147484e+09
3,30.0,1970-01-01 00:00:30,1970-01-01,00:00:30,-80.330711,-122.467491,0.0000,9.969210e+36,9.969210e+36,9.969210e+36,9.969210e+36,9.969210e+36,-2.147484e+09
4,30.0,1970-01-01 00:00:30,1970-01-01,00:00:30,-80.371796,-122.911118,0.0000,9.969210e+36,9.969210e+36,9.969210e+36,9.969210e+36,9.969210e+36,-2.147484e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1680745,46.0,1970-01-01 00:00:46,1970-01-01,00:00:46,85.877251,71.582901,0.0007,-3.840074e-05,2.800259e-05,2.692881e-05,2.510756e+00,1.205406e+01,1.300000e+01
1680746,46.0,1970-01-01 00:00:46,1970-01-01,00:00:46,85.813965,72.241875,0.0007,-4.188190e-05,2.857801e-05,2.732287e-05,2.474772e+00,1.210247e+01,1.300000e+01
1680747,46.0,1970-01-01 00:00:46,1970-01-01,00:00:46,85.749306,72.889397,0.0007,-4.467173e-05,2.973610e-05,2.836218e-05,2.420678e+00,1.215186e+01,1.300000e+01
1680748,46.0,1970-01-01 00:00:46,1970-01-01,00:00:46,85.683243,73.525665,0.0007,-5.909920e-05,3.172470e-05,2.944058e-05,2.345738e+00,1.220259e+01,1.300000e+01


In [ ]:
df=nc_to_df(allFiles[0])

In [58]:
df[df['nitrogendioxide_tropospheric_column']!= Data.groups['PRODUCT'].variables['nitrogendioxide_tropospheric_column']._FillValue]

,UnixTimestamp,DateTime,Date,Time,latitude,longitude,qa_value,nitrogendioxide_tropospheric_column,nitrogendioxide_tropospheric_column_precision,nitrogendioxide_tropospheric_column_precision_kernel,air_mass_factor_troposphere,air_mass_factor_total,tm5_tropopause_layer_index
111601,58.0,1970-01-01 00:00:58,1970-01-01,00:00:58,-85.237633,-19.944014,0.0007,-0.000030,0.000020,0.000019,5.647959,14.558993,12.0
112051,59.0,1970-01-01 00:00:59,1970-01-01,00:00:59,-85.209114,-19.493507,0.0007,-0.000020,0.000018,0.000018,5.799623,14.446025,12.0
112052,59.0,1970-01-01 00:00:59,1970-01-01,00:00:59,-85.269310,-18.730186,0.0007,-0.000027,0.000019,0.000018,5.818497,14.473722,12.0
112053,59.0,1970-01-01 00:00:59,1970-01-01,00:00:59,-85.327843,-17.958418,0.0007,-0.000022,0.000018,0.000018,5.833448,14.508048,12.0
112054,59.0,1970-01-01 00:00:59,1970-01-01,00:00:59,-85.384720,-17.178270,0.0007,-0.000024,0.000019,0.000018,5.848399,14.543132,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1680745,46.0,1970-01-01 00:00:46,1970-01-01,00:00:46,85.877251,71.582901,0.0007,-0.000038,0.000028,0.000027,2.510756,12.054061,13.0
1680746,46.0,1970-01-01 00:00:46,1970-01-01,00:00:46,85.813965,72.241875,0.0007,-0.000042,0.000029,0.000027,2.474772,12.102472,13.0
1680747,46.0,1970-01-01 00:00:46,1970-01-01,00:00:46,85.749306,72.889397,0.0007,-0.000045,0.000030,0.000028,2.420678,12.151863,13.0
1680748,46.0,1970-01-01 00:00:46,1970-01-01,00:00:46,85.683243,73.525665,0.0007,-0.000059,0.000032,0.000029,2.345738,12.202587,13.0


In [111]:
df_all=pd.DataFrame()
for i in tqdm(allFiles):
    df=nc_to_df(i)
    df_all.append(df)

  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

BlueSkyChallenge-main\data\L2\S5P_OFFL_L2__NO2____20210325T104955_20210325T123125_17863_01_010400_20210327T044257.nc is a TROPOMI NO2 file.
latitude (1, 3735, 450)
longitude (1, 3735, 450)
qa_value (1, 3735, 450)
nitrogendioxide_tropospheric_column (1, 3735, 450)
nitrogendioxide_tropospheric_column_precision (1, 3735, 450)
nitrogendioxide_tropospheric_column_precision_kernel (1, 3735, 450)
air_mass_factor_troposphere (1, 3735, 450)
air_mass_factor_total (1, 3735, 450)
tm5_tropopause_layer_index (1, 3735, 450)
Processed BlueSkyChallenge-main\data\L2\S5P_OFFL_L2__NO2____20210325T104955_20210325T123125_17863_01_010400_20210327T044257.nc in 0.19930159499999717 minutes


  5%|████▏                                                                              | 1/20 [00:12<03:56, 12.43s/it]

BlueSkyChallenge-main\data\L2\S5P_OFFL_L2__NO2____20210325T123125_20210325T141256_17864_01_010400_20210327T060331.nc is a TROPOMI NO2 file.
latitude (1, 4173, 450)
longitude (1, 4173, 450)
qa_value (1, 4173, 450)
nitrogendioxide_tropospheric_column (1, 4173, 450)
nitrogendioxide_tropospheric_column_precision (1, 4173, 450)
nitrogendioxide_tropospheric_column_precision_kernel (1, 4173, 450)
air_mass_factor_troposphere (1, 4173, 450)
air_mass_factor_total (1, 4173, 450)
tm5_tropopause_layer_index (1, 4173, 450)
Processed BlueSkyChallenge-main\data\L2\S5P_OFFL_L2__NO2____20210325T123125_20210325T141256_17864_01_010400_20210327T060331.nc in 0.22440753000000768 minutes


 10%|████████▎                                                                          | 2/20 [00:26<04:01, 13.42s/it]

...

 90%|█████████████████████████████████████████████████████████████████████████▊        | 18/20 [04:24<00:30, 15.02s/it]

BlueSkyChallenge-main\data\L2\S5P_OFFL_L2__NO2____20210404T110345_20210404T124515_18005_01_010400_20210406T043200.nc is a TROPOMI NO2 file.
latitude (1, 4173, 450)
longitude (1, 4173, 450)
qa_value (1, 4173, 450)
nitrogendioxide_tropospheric_column (1, 4173, 450)
nitrogendioxide_tropospheric_column_precision (1, 4173, 450)
nitrogendioxide_tropospheric_column_precision_kernel (1, 4173, 450)
air_mass_factor_troposphere (1, 4173, 450)
air_mass_factor_total (1, 4173, 450)
tm5_tropopause_layer_index (1, 4173, 450)
Processed BlueSkyChallenge-main\data\L2\S5P_OFFL_L2__NO2____20210404T110345_20210404T124515_18005_01_010400_20210406T043200.nc in 0.22780056166666326 minutes


 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [04:38<00:14, 14.82s/it]

BlueSkyChallenge-main\data\L2\S5P_OFFL_L2__NO2____20210404T124515_20210404T142646_18006_01_010400_20210406T062009.nc is a TROPOMI NO2 file.
latitude (1, 4173, 450)
longitude (1, 4173, 450)
qa_value (1, 4173, 450)
nitrogendioxide_tropospheric_column (1, 4173, 450)
nitrogendioxide_tropospheric_column_precision (1, 4173, 450)
nitrogendioxide_tropospheric_column_precision_kernel (1, 4173, 450)
air_mass_factor_troposphere (1, 4173, 450)
air_mass_factor_total (1, 4173, 450)
tm5_tropopause_layer_index (1, 4173, 450)
Processed BlueSkyChallenge-main\data\L2\S5P_OFFL_L2__NO2____20210404T124515_20210404T142646_18006_01_010400_20210406T062009.nc in 0.23846432333333117 minutes


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [04:53<00:00, 14.70s/it]


In [18]:
f['groups'

<bound method Mapping.keys of <xarray.Dataset>
Dimensions:  ()
Data variables:
    *empty*
Attributes: (12/53)
    Conventions:                        CF-1.7
    institution:                        KNMI
    source:                             Sentinel 5 precursor, TROPOMI, space-...
    summary:                            TROPOMI/S5P NO2 1-Orbit L2 Swath 7x3.5km
    tracking_id:                        67b8f02c-12c2-45cc-bd13-9b44a17bd568
    id:                                 S5P_OFFL_L2__NO2____20210325T104955_2...
    ...                                 ...
    Status_MET_2D:                      Nominal
    Status_NISE__:                      Nominal
    Status_CTMFCT:                      Nominal
    history:                            2021-03-27 04:54:57 f_s5pops tropnll2...
    processing_status:                  OFFL-processing nominal product
    date_modified:                      2021-04-02T13:12:09Z>

In [41]:

where_j = np.where((lons >= -0.6) & (lons <= 0.28))[0]
where_i = np.where((lats >= 51.25) & (lats <= 52.75))[0]

i0 = where_i[0]
i1 = where_i[-1]+1

j0 = where_j[0]
j1 = where_j[-1]+1

print(lats[i0:i1])  

[[45. 45. 45. ... 51. 51. 51.]
 [45. 45. 45. ... 51. 51. 51.]
 [45. 45. 45. ... 51. 51. 51.]
 ...
 [52. 52. 53. ... 60. 60. 60.]
 [52. 52. 53. ... 60. 60. 60.]
 [52. 53. 53. ... 60. 60. 60.]]


In [42]:
where_i

array([3007, 3007, 3007, ..., 3187, 3187, 3188], dtype=int64)